# 1D Maze Results Viewer

Load saved `.pcellbundle` results from 1D maze analysis and inspect them
without re-running the pipeline. Supports loading a single bundle or
multiple bundles for cross-session comparison.

---

**Note:** Interactive widgets require **Jupyter Lab**:

```bash
cd notebook && jupyter lab --no-browser --port=6006
```

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

from placecell.dataset import BasePlaceCellDataset
from placecell.notebook import browse_units_1d, create_shuffle_browser_1d
from placecell.visualization import (
    plot_diagnostics,
    plot_footprints_filled,
    plot_graph_overlay,
    plot_occupancy_preview_1d,
    plot_position_and_traces_1d,
    plot_session_summary,
    plot_shuffle_test_1d,
    plot_summary_scatter,
)

## Load Bundles

List the `.pcellbundle` paths to load. For a single session, use one entry.

In [ ]:
BUNDLE_PATHS = [
    Path("../user_data/bundles/WL25_20251219_no_partial.pcellbundle"),
]

datasets = {}
for bp in BUNDLE_PATHS:
    name = bp.stem.replace(".pcellbundle", "")
    ds = BasePlaceCellDataset.load_bundle(bp)
    datasets[name] = ds
    s = ds.summary()
    print(
        f"{name}: {s['n_total']} units, "
        f"{s['n_sig']} sig, {s['n_stable']} stable, "
        f"{s['n_place_cells']} place cells"
    )

## Cross-Session Summary

Counts and proportions of significant, stable, and place cell units across sessions.
Only shown when multiple bundles are loaded.

In [ ]:
rows = []
for name, ds in datasets.items():
    rows.append({"dataset": name, **ds.summary()})
summary_df = pd.DataFrame(rows)
display(summary_df)

if len(datasets) > 1:
    plot_session_summary(summary_df)
    plt.show()

---

## Per-Session Results

Select a session to inspect. Change `SESSION` to switch.

In [ ]:
SESSION = list(datasets.keys())[0]
ds = datasets[SESSION]
p_thresh = ds.spatial_1d.p_value_threshold
print(f"Session: {SESSION}")
print(
    f"Config: bin_width_mm={ds.spatial_1d.bin_width_mm}, "
    f"spatial_sigma={ds.spatial_1d.spatial_sigma}, "
    f"n_shuffles={ds.spatial_1d.n_shuffles}, "
    f"p_threshold={p_thresh}"
)

### Behavior Graph Overlay

Zone polylines from the behavior graph overlaid on the video frame.

In [ ]:
if ds.graph_polylines is not None:
    plot_graph_overlay(
        ds.graph_polylines,
        ds.graph_mm_per_pixel,
        arm_order=ds.maze_cfg.arm_order,
        video_frame=ds.behavior_video_frame,
    )
    plt.show()
else:
    print("No behavior graph available in this bundle.")

### Max Projection + Spatial Footprints

In [ ]:
if ds.max_proj is not None and ds.footprints is not None:
    plot_footprints_filled(ds.max_proj, ds.footprints)
    plt.show()
else:
    print("Max projection or footprints not available in this bundle.")

### Occupancy

In [ ]:
if ds.occupancy_time is not None and ds.edges_1d is not None:
    plot_occupancy_preview_1d(
        ds.trajectory_1d_filtered,
        ds.occupancy_time,
        ds.valid_mask,
        ds.edges_1d,
        trajectory_1d=ds.trajectory_1d,
        trajectory_1d_all=getattr(ds, "trajectory_1d_all", None),
        arm_boundaries=ds.arm_boundaries,
        arm_labels=ds.effective_arm_order,
    )
    plt.show()
else:
    print("Occupancy data not available in this bundle.")

### Position + Place Cell Traces

Serialized 1D position and 25 example place cell calcium traces, time-synced.
Cells are sorted by peak firing position.

In [ ]:
place_cell_results = ds.place_cells()
if place_cell_results and ds.trajectory_1d is not None:
    plot_position_and_traces_1d(
        ds.trajectory_1d,
        place_cell_results,
        ds.edges_1d,
        behavior_fps=ds.cfg.behavior.behavior_fps,
        speed_threshold=ds.cfg.behavior.speed_threshold,
        trajectory_1d_filtered=ds.trajectory_1d_filtered,
        arm_boundaries=ds.arm_boundaries,
        arm_labels=ds.effective_arm_order,
    )
    plt.show()
else:
    print("No place cells or trajectory data available.")

### Diagnostics

Event count distribution, SI vs event count, and p-value vs event count.

In [ ]:
plot_diagnostics(ds.unit_results, p_value_threshold=p_thresh)
plt.show()

### Significance vs Stability

In [ ]:
plot_summary_scatter(
    ds.unit_results, p_value_threshold=p_thresh,
    n_shuffles=ds.spatial_1d.n_shuffles,
    min_shift_seconds=ds.spatial_1d.min_shift_seconds,
)
plt.show()

### Unit Summary Table

Sortable table with key metrics for all units.

In [ ]:
table_rows = []
for uid, res in sorted(ds.unit_results.items()):
    is_sig = res.p_val < p_thresh
    is_stable = (
        not np.isnan(res.stability_p_val) and res.stability_p_val < p_thresh
    )
    stab_r = (
        round(res.stability_corr, 4) if np.isfinite(res.stability_corr) else np.nan
    )
    stab_z = (
        round(res.stability_z, 4) if np.isfinite(res.stability_z) else np.nan
    )
    stab_p = (
        round(res.stability_p_val, 4)
        if np.isfinite(res.stability_p_val)
        else np.nan
    )
    table_rows.append({
        "unit_id": uid,
        "n_events": len(res.unit_data),
        "SI (bits/spike)": round(res.si, 4),
        "p_val": round(res.p_val, 4),
        "sig": is_sig,
        "stability_r": stab_r,
        "stability_z": stab_z,
        "stability_p": stab_p,
        "stable": is_stable,
        "place_cell": is_sig and is_stable,
    })

unit_table = pd.DataFrame(table_rows).set_index("unit_id")
display(unit_table)

### Population Rate Map

Rate maps of all place cells sorted by peak position.

In [ ]:
plot_shuffle_test_1d(
    ds.unit_results,
    ds.edges_1d,
    p_value_threshold=p_thresh,
    arm_boundaries=ds.arm_boundaries,
    arm_labels=ds.effective_arm_order,
)
plt.show()

### Per-Unit Shuffle Browser

Browse each unit's rate map, SI shuffle distribution, and stability shuffle distribution.

In [ ]:
%matplotlib widget

fig_shuf, controls_shuf = create_shuffle_browser_1d(
    ds.unit_results,
    ds.edges_1d,
    p_value_threshold=p_thresh,
    arm_boundaries=ds.arm_boundaries,
    arm_labels=ds.effective_arm_order,
)
plt.show()
display(controls_shuf)

### Interactive Unit Browser

Browse individual units: rate maps (1st half / 2nd half / full), shuffle histograms, and calcium trace with events.

In [ ]:
%matplotlib widget

fig_units, controls_units = browse_units_1d(ds)
plt.show()
display(controls_units)

### Place Cell Browser (Significant AND Stable Only)

In [ ]:
%matplotlib widget

place_cell_results = ds.place_cells()
n_place_cells = len(place_cell_results)
print(f"Place cells (sig + stable): {n_place_cells} / {len(ds.unit_results)}")

if n_place_cells > 0:
    fig_pc, controls_pc = browse_units_1d(ds, unit_results=place_cell_results)
    plt.show()
    display(controls_pc)
else:
    print("No cells passed both significance and stability tests.")